#Chapter 16

Word2Vec

Train classifer on a binary prediction task

Self-supervision sample data taken from the corpus

see if something is good or bad


Negative Sampling
p(Ci is context |W) 1 or 0

Skip Gram Negative Sampling (SGNS)

use a +/- 2 window

need to create a negative pair at least k amount (k = 4)
this can be done by taking the word and randomly selecting a word



In [ ]:
from sklearn.datasets import fetch_20newsgroups as getData
import numpy as np
import re
from collections import defaultdict

X,y = getData(subset='train',remove=('headers','footers','quotes'),return_X_y=True)

print(X)
print(X[0])

#get all tokens and there frequence
fr = defaultdict(int)
for doc in X:
    for token in re.split('\W+',doc.lower()):
        fr[token] += 1

In [3]:
#any token appears less than 10 times remove
cutOffValue = 10
processed_corpus = []
for doc in X:
    for token in re.split('\W+',doc.lower()):
        if fr[token] >= cutOffValue:
            processed_corpus.append(token)


allWords = np.array(list(fr.keys()))#all words
allCounts = np.array(list(fr.values()))#there counts
vocab = allWords[allCounts >= cutOffValue]#all words having counts > 10
wordCounts = allCounts[allCounts >= cutOffValue]#all counts > 10



alpha = 0.75 #alpha for negative sampling
wordCounts = wordCounts**alpha #help array to increase prop
probs = wordCounts/np.sum(wordCounts)#all probs

numWords = len(vocab)
W2I = dict(zip(vocab,np.arange(numWords)))#build model of the word and count
I2W = dict(zip(np.arange(numWords),vocab))#reverse of the above

#
def sigmoid(x):
    return 1/(1+np.exp(-x))

#
def gradientStep(w,cPos,cNeg,lr=0.001):
    cPos_new = cPos - lr*(sigmoid(w.dot(cPos))-1)*w
    w_new = w - lr*(sigmoid(w.dot(cPos))-1)*cPos
    cNeg_new = np.zeros(cNeg.shape)
    for i in range(cNeg.shape[0]):
        v = sigmoid(cNeg[i,:].dot(w))
        w_new -= lr*v*cNeg[i,:]
        cNeg_new[i,:] = cNeg[i,:] - lr*v*w
    return w_new,cPos_new,cNeg_new

#W=init embedings
#C=context embedings
#doc=document in list form
#vocab=vocabulary
#W2I=word to index mapping
#I2W=index to word mapping
#probs= probabilites of vocabulary
#winSize=window size
#k=
def word2vecSGNS(W,C,doc,vocab,W2I,I2W,probs,winSize=2,k=4):
    numTokens = len(doc) #len of dcoument
    for i in range(numTokens):
        for j in range(i-winSize,i+winSize):#
            if j != i and j >= 0 and j < numTokens:
                wIdx = W2I[doc[i]]
                posIdx = W2I[doc[j]]
                w = W[wIdx,:]
                cPos = C[posIdx,:]#this is one pair of many
                cNeg = np.zeros((k,C.shape[1]))
                m = 0
                negIdx = []
                for negC in np.random.choice(list(vocab),k,list(probs)):
                    negIdx.append(W2I[negC])
                    cNeg[m,:] = C[W2I[negC],:]
                    m += 1
        w_new,cPos_new,cNeg_new = gradientStep(w,cPos,cNeg)
        W[wIdx,:] = w_new
        C[posIdx,:] = cPos_new
        C[negIdx,:] = cNeg_new
    return W,C

W = np.random.rand(numWords,100)
C = np.random.rand(numWords,100)
W,C = word2vecSGNS(W,C,processed_corpus,vocab,W2I,I2W,probs)

KeyboardInterrupt: 